In [1]:
import sqlite3
import json
import pandas as pd
pd.set_option('display.max_colwidth',1000)

## Create db with exoplanets data

- Data source: https://www.kaggle.com/datasets/adityamishraml/nasaexoplanets

In [15]:
# exoplanets_df = pd.read_csv('cleaned_5250.csv')
# conn = sqlite3.connect('./databases/exoplanets_db.db')
# exoplanets_df.to_sql('exoplanets',conn,if_exists='replace',index=False)
# conn.close()

In [21]:
conn = sqlite3.connect('./databases/exoplanets_db.db')
cursor = conn.cursor()
cursor.execute(f"SELECT * FROM exoplanets LIMIT 5;")

In [22]:
for row in cursor:
    print(row)

('11 Comae Berenices b', 304.0, 4.72307, 'Gas Giant', 2007, 19.4, 'Jupiter', 1.08, 'Jupiter', 1.29, 0.8925394, 0.23, 'Radial Velocity')
('11 Ursae Minoris b', 409.0, 5.013, 'Gas Giant', 2009, 14.74, 'Jupiter', 1.09, 'Jupiter', 1.53, 1.4, 0.08, 'Radial Velocity')
('14 Andromedae b', 246.0, 5.23133, 'Gas Giant', 2008, 4.8, 'Jupiter', 1.15, 'Jupiter', 0.83, 0.5086927, 0.0, 'Radial Velocity')
('14 Herculis b', 58.0, 6.61935, 'Gas Giant', 2002, 8.13881, 'Jupiter', 1.12, 'Jupiter', 2.773069, 4.8, 0.37, 'Radial Velocity')
('16 Cygni B b', 69.0, 6.215, 'Gas Giant', 1996, 1.78, 'Jupiter', 1.2, 'Jupiter', 1.66, 2.2, 0.68, 'Radial Velocity')


In [23]:
cursor.execute("PRAGMA table_info(exoplanets)")

In [24]:
for row in cursor:
    print(row)

(0, 'name', 'TEXT', 0, None, 0)
(1, 'distance', 'REAL', 0, None, 0)
(2, 'stellar_magnitude', 'REAL', 0, None, 0)
(3, 'planet_type', 'TEXT', 0, None, 0)
(4, 'discovery_year', 'INTEGER', 0, None, 0)
(5, 'mass_multiplier', 'REAL', 0, None, 0)
(6, 'mass_wrt', 'TEXT', 0, None, 0)
(7, 'radius_multiplier', 'REAL', 0, None, 0)
(8, 'radius_wrt', 'TEXT', 0, None, 0)
(9, 'orbital_radius', 'REAL', 0, None, 0)
(10, 'orbital_period', 'REAL', 0, None, 0)
(11, 'eccentricity', 'REAL', 0, None, 0)
(12, 'detection_method', 'TEXT', 0, None, 0)


In [ ]:
cursor.execute("""
SELECT discovery_year, AVG("Mass (MJ)") AS average_mass
FROM exoplanets
GROUP BY "Disc. Year"
ORDER BY "Disc. Year";
""")

In [11]:
# query = "CREATE TABLE reference_planets (name TEXT, mass REAL);"

# cursor.execute(query)

In [ ]:
#cursor.execute("DELETE  from reference_planets where 1==1;")

In [30]:
# insert1 = """
# INSERT INTO reference_planets (name, mass) VALUES ('Jupiter', 1.898e27);
# """
# insert2 = """
# INSERT INTO reference_planets (name, mass) VALUES ('Earth', 5.972e24);
# """
# cursor.execute(insert1);cursor.execute(insert2)
# conn.commit()
# conn.close()

In [ ]:
cursor.execute("SELECT * FROM reference_planets;")
for row in cursor:
    print(row)

In [4]:
def test_query(query,verbose=False):
    cursor.execute(query)
    if verbose:
        for row in cursor:
            print(row)

In [ ]:
#How many exoplanets were discovered using each method? (Order by the number of discoveries)
query=f"""
SELECT 
    e.detection_method,
    COUNT(*) AS count_exoplanets
FROM exoplanets e
GROUP BY e.detection_method
ORDER BY count_exoplanets DESC;  
"""
test_query(query,verbose=True)

In [54]:
#Which exoplanets are located in the habitable zone (typically 0.95 to 1.37 AU for stars similar to the Sun)?
query=f"""
SELECT 
    e.name,
    e.orbital_radius
FROM exoplanets e
WHERE e.orbital_radius BETWEEN 0.95 AND 1.37;
"""
test_query(query)

In [55]:
#List exoplanets that are within 20 light-years of Earth.
query=f"""
SELECT 
    e.name,
    e.distance
FROM exoplanets e
WHERE e.distance < 20;  -- Within 20 light-years
"""
test_query(query)

In [ ]:
exoplanets_df[exoplanets_df.name=='11 Comae Berenices b']

In [57]:
# Find exoplanets with eccentricity values greater than 0.5, indicating more elongated orbits. Order by highest eccentricity.
query=f"""
SELECT 
    e.name,
    e.eccentricity,
    e.orbital_period
FROM exoplanets e
WHERE e.eccentricity > 0.5
ORDER BY e.eccentricity DESC;  -- Order by highest eccentricity
"""
test_query(query)

In [58]:
# Find exoplanets with mass greater than a certain threshold (e.g., 5 MJ).
query=f"""
WITH MassFactors AS (
    SELECT 
        MAX(CASE WHEN name = 'Earth' THEN mass END) AS earth_mass,
        MAX(CASE WHEN name = 'Jupiter' THEN mass END) AS jupiter_mass
    FROM reference_planets
)

SELECT 
    e.name,
    CASE 
        WHEN e.mass_wrt = 'Earth' THEN e.mass_multiplier * r.mass  -- Mass in Earth masses
        WHEN e.mass_wrt = 'Jupiter' THEN e.mass_multiplier * r.mass * (SELECT jupiter_mass / earth_mass FROM MassFactors)  -- Convert to Earth masses
    END AS mass_in_earth_masses
FROM exoplanets e
JOIN reference_planets r ON e.mass_wrt = r.name
WHERE (CASE 
            WHEN e.mass_wrt = 'Earth' THEN e.mass_multiplier * r.mass
            WHEN e.mass_wrt = 'Jupiter' THEN e.mass_multiplier * r.mass * (SELECT jupiter_mass / earth_mass FROM MassFactors)
        END) > 5;-- Mass threshold in Earth masses
"""
test_query(query)

In [59]:
# Find exoplanets with mass greater than 5 Earth masses
query=f"""
WITH MassFactors AS (
    SELECT 
        MAX(CASE WHEN name = 'Earth' THEN mass END) AS earth_mass,
        MAX(CASE WHEN name = 'Jupiter' THEN mass END) AS jupiter_mass
    FROM reference_planets
)

SELECT 
    e.name,
    CASE 
        WHEN e.mass_wrt = 'Earth' THEN e.mass_multiplier * r.mass  -- Mass in Earth masses
        WHEN e.mass_wrt = 'Jupiter' THEN e.mass_multiplier * r.mass * (SELECT jupiter_mass / earth_mass FROM MassFactors)  -- Convert to Earth masses
    END AS mass_in_earth_masses
FROM exoplanets e
JOIN reference_planets r ON e.mass_wrt = r.name
WHERE (CASE 
            WHEN e.mass_wrt = 'Earth' THEN e.mass_multiplier * r.mass
            WHEN e.mass_wrt = 'Jupiter' THEN e.mass_multiplier * r.mass * (SELECT jupiter_mass / earth_mass FROM MassFactors)
        END) > 5;  -- Mass threshold in Earth masses
  -- Mass threshold in Earth masses
"""
test_query(query)

In [ ]:
19.4*1.898e+27*(1.898e+27/5.972e+24) # perhaps was overestimated in db!!! query correct,though
#('Jupiter', 1.898e+27)
#('Earth', 5.972e+24)

In [61]:
# Find the 5 closest exoplanets that are also the brightest.
query=f"""
SELECT 
    e.name,
    e.distance,
    e.stellar_magnitude
FROM exoplanets e
WHERE e.distance IS NOT NULL and e.stellar_magnitude IS NOT NULL
ORDER BY e.distance ASC, e.stellar_magnitude ASC  -- Closest and brightest
LIMIT 5;
"""
test_query(query)

In [62]:
# Calculate the average orbital radius for each type of exoplanet. Order by average orbital radius descending.
query=f"""
SELECT 
    e.planet_type,
    AVG(e.orbital_radius) AS avg_orbital_radius
FROM exoplanets e
GROUP BY e.planet_type
ORDER BY avg_orbital_radius DESC;  
"""
test_query(query)

In [63]:
# Count how many exoplanets of each type were discovered each year. Order by year ascending and count descending.
query=f"""
SELECT 
    e.discovery_year,
    e.planet_type,
    COUNT(*) AS count
FROM exoplanets e
GROUP BY e.discovery_year, e.planet_type
ORDER BY e.discovery_year, count DESC;  
"""
test_query(query)

In [64]:
# Find exoplanets with an eccentricity greater than 0.6 and an orbital period longer than 200 days.
query=f"""
SELECT 
    e.name,
    e.eccentricity,
    e.orbital_period
FROM exoplanets e
WHERE e.eccentricity > 0.6 AND e.orbital_period > 200
ORDER BY e.eccentricity DESC; 
"""
test_query(query)

In [65]:
# List all exoplanets discovered in the last 5 years, sorted by discovery year.
query=f"""
SELECT 
    e.name,
    e.discovery_year
FROM exoplanets e
WHERE e.discovery_year >= strftime('%Y', 'now') - 5  -- Last 5 years
ORDER BY e.discovery_year DESC;  -- Most recent first
"""
test_query(query)

In [66]:
#  Identify the most common types of exoplanets among the top 10 brightest, order by frequency of planet types.
query=f"""
WITH BrightestExoplanets AS (
    SELECT 
        e.name,
        e.planet_type,
        e.stellar_magnitude
    FROM exoplanets e
    ORDER BY e.stellar_magnitude ASC  -- Brightest first
    LIMIT 10
)

SELECT 
    planet_type,
    COUNT(*) AS count
FROM BrightestExoplanets
GROUP BY planet_type
ORDER BY count DESC;
"""
test_query(query)

In [67]:
# Analyze the relationship between the discovery year and the average stellar magnitude of exoplanets discovered each year.
query=f"""
SELECT 
    e.discovery_year,
    AVG(e.stellar_magnitude) AS avg_stellar_magnitude
FROM exoplanets e
GROUP BY e.discovery_year
ORDER BY e.discovery_year;  -- Order by discovery year

"""
test_query(query)

In [68]:
test_query("""
SELECT planet_type, name, radius_multiplier
FROM exoplanets
WHERE (planet_type, radius_multiplier) IN (
    SELECT planet_type, radius_multiplier
    FROM exoplanets
    ORDER BY planet_type, radius_multiplier DESC
)
GROUP BY planet_type
LIMIT 2;
""")

In [69]:
test_query("""
SELECT discovery_year, 
SUM(mass_multiplier * (SELECT mass FROM reference_planets WHERE name = mass_wrt)) AS total_mass
FROM exoplanets
WHERE mass_multiplier IS NOT NULL
GROUP BY discovery_year
ORDER BY total_mass DESC
LIMIT 5;
""")

## Testing query results in batch

In [22]:
generated_sql_df = pd.read_csv('Exoplanets-OG20 - Sheet1.csv')

In [ ]:
generated_sql_df.head(2)

In [ ]:
sql_queries=list(generated_sql_df['answer'])
len(sql_queries)

In [25]:
error_queries = []

# Process each query
for idx, query in enumerate(sql_queries):
    try:
        test_query(query)
    except Exception as e:
        print(f"Error occurred: {e}")
        error_queries.append({'query': query, 'idx': idx, 'exception': type(e).__name__})
        


In [ ]:
error_queries

In [ ]:
test_query('SELECT discovery_year, name, orbital_period FROM exoplanets WHERE (discovery_year, name, orbital_period) IN (   SELECT discovery_year, name, orbital_period    FROM exoplanets    ORDER BY discovery_year, orbital_period DESC) GROUP BY discovery_year LIMIT 3;', verbose=True)

In [ ]:
test_query('SELECT e.name, e.orbital_radius, e.mass_multiplier * r.mass AS mass FROM exoplanets e JOIN reference_planets r ON e.mass_wrt = r.name WHERE e.orbital_radius BETWEEN 0.5 AND 1.5  ORDER BY mass DESC;', verbose=True)

In [ ]:
test_query("""WITH DetectionTrends AS (
    SELECT 
        discovery_year,
        detection_method,
        COUNT(*) AS method_count
    FROM exoplanets
    GROUP BY discovery_year, detection_method
),
RankedTrends AS (
    SELECT 
        discovery_year,
        detection_method,
        method_count, RANK() OVER (PARTITION BY discovery_year ORDER BY method_count DESC) AS method_rank FROM DetectionTrends) SELECT  discovery_year, detection_method, method_count, method_rank FROM RankedTrends WHERE method_rank <= 2 ORDER BY discovery_year, method_rank;""", verbose=True)

In [5]:
def test_rows(query):
    cursor.execute(query)
    rows = cursor.fetchall()
    if not rows:
        raise ValueError(f"The following query returned no rows: {query}")
    

In [31]:
for idx, query in enumerate(sql_queries):
    try:
        test_rows(query)
    except Exception as e:
        print(f"Look at idx: {idx+1}")
        print(e)

In [ ]:
test_query("""
WITH detection_counts AS (
    SELECT 
        detection_method, 
        COUNT(*) AS method_count
    FROM 
        exoplanets
    GROUP BY 
        detection_method
),
least_common_detection AS (
    SELECT 
        detection_method
    FROM 
        detection_counts
    WHERE 
        method_count = (SELECT MIN(method_count) FROM detection_counts)
)
SELECT 
    e.*
FROM 
    exoplanets e
JOIN 
    least_common_detection lcd ON e.detection_method = lcd.detection_method;
           """, verbose=True)

In [17]:
test_rows("""
WITH detection_counts AS (
    SELECT 
        detection_method, 
        COUNT(*) AS method_count
    FROM 
        exoplanets
    GROUP BY 
        detection_method
),
least_common_detection AS (
    SELECT 
        detection_method
    FROM 
        detection_counts
    WHERE 
        method_count = (SELECT MIN(method_count) FROM detection_counts)
)
SELECT 
    e.*
FROM 
    exoplanets e
JOIN 
    least_common_detection lcd ON e.detection_method = lcd.detection_method;
           """)

In [19]:
# def run_query(query, cursor):
#     cursor.execute(query)
#     rows = cursor.fetchall()
#     columns = [desc[0] for desc in cursor.description]
#     return pd.DataFrame(rows, columns=columns)

def run_query(query, cursor):
    cursor.execute(query)
    rows = cursor.fetchall()
    columns = [desc[0] for desc in cursor.description]
    result_dict = [dict(zip(columns, row)) for row in rows]
    return json.dumps(result_dict)

In [ ]:
run_query(sql_queries[0],cursor)

In [ ]:
sql_queries[0]

In [34]:
generated_sql_df['results'] = generated_sql_df['answer'].map(lambda query: run_query(query,cursor))

In [ ]:
generated_sql_df.head()

In [35]:
from sklearn.model_selection import train_test_split
shuffled_df = generated_sql_df.sample(frac=1,random_state=42).reset_index(drop=True)#.rename(columns={'answer':'SQL'})

In [36]:
 # roll the dice via random seed but use judgement to get representative set 
train_df,valid_df = train_test_split(shuffled_df,test_size=10,random_state=177)

In [ ]:
valid_df

In [111]:
#valid_df.rename(columns={'answer':'SQL'},inplace=True)
#train_df.rename(columns={'answer':'SQL'},inplace=True)
#train_df.to_json('./databases/train.json',orient='records',lines=True)
#valid_df.to_json('./databases/validation.json',orient='records',lines=True)

In [38]:
# Convert the DataFrame to a list of dictionaries
train_data = train_df.to_dict(orient='records')
valid_data = valid_df.to_dict(orient='records')

# Save the list of dictionaries to a JSON file
with open('./databases/train.json', 'w') as train_file:
    json.dump(train_data, train_file, indent=4)

with open('./databases/valid.json', 'w') as valid_file:
    json.dump(valid_data, valid_file, indent=4)

# # Verify the saved JSON files
# with open('./databases/train.json', 'r') as train_file:
#     train_loaded = json.load(train_file)
#     print("Train Data Loaded:")
#     print(train_loaded[:2])  # Print the first 2 records for verification

# with open('./databases/valid.json', 'r') as valid_file:
#     valid_loaded = json.load(valid_file)
#     print("Valid Data Loaded:")
#     print(valid_loaded[:2])  # Print the first 2 records for verification

In [ ]:
# make sure to rename above if don't go building own evaluators approach fully
from premsql.datasets import StandardDataset

validation_dataset = StandardDataset(
    split="validation",    # it can be either train / validation / test depending on your dataset and the name of the json file
    dataset_path="./databases",
    database_folder_name="databases", # The same name of the folder 
    json_file_name="valid.json",
)

train_dataset = StandardDataset(
    split="train",    # it can be either train / validation / test depending on your dataset and the name of the json file
    dataset_path="./databases",
    database_folder_name="databases", # The same name of the folder 
    json_file_name="train.json",
)


In [ ]:
train_dataset

### Format as HF datasets

In [39]:
from datasets import load_dataset, DatasetDict

In [ ]:
train_dataset = load_dataset('json', data_files='./databases/train.json', split='train')
valid_dataset = load_dataset('json', data_files='./databases/valid.json', split='train')


In [41]:
dataset = DatasetDict({
    'train': train_dataset,
    'valid': valid_dataset
}
)

In [ ]:
dataset

In [ ]:
#!huggingface-cli login
dataset.push_to_hub('dpv/exoplanets-sql')

In [45]:
from huggingface_hub import HfApi, Repository
api = HfApi()

- Tried to update README.MD multiple times with the following:
This is a small dataset based on https://www.kaggle.com/datasets/adityamishraml/nasaexoplanets/data.  sqlite table *exoplanets* was made from the data, along with a *reference_planets* table made by inserting  (name, mass) VALUES ('Jupiter', 1.898e27) and  (name, mass) VALUES ('Earth', 5.972e24).
The *mass_wrt* column in *exoplanets* maps to *mass* table in *reference_planets*.  This table allows for more complex queries involving joins.  Queries have been checked for logical consistency, as well as by running against the database.

In [ ]:
with open('./hf_readme.md', "r") as file:
    readme_content = file.read()

# Update the README.md file on the Hugging Face Hub
api.upload_file(
    path_or_fileobj='./hf_readme.md',
    path_in_repo="README.md",
    repo_id='dpv/exoplanets-sql',
    repo_type="dataset"
)

print("Updated README.md")

## Experiments

In [52]:
query1 = """WITH DetectionTrends AS (
    SELECT 
        discovery_year,
        detection_method,
        COUNT(*) AS method_count
    FROM exoplanets
    GROUP BY discovery_year, detection_method
),
RankedTrends AS (
    SELECT 
        discovery_year,
        detection_method,
        method_count, RANK() OVER (PARTITION BY discovery_year ORDER BY method_count DESC) AS method_rank FROM DetectionTrends) SELECT  discovery_year, detection_method, method_count, method_rank FROM RankedTrends WHERE method_rank <= 2 ORDER BY discovery_year, method_rank;"""

query2 = """WITH DetectionTrends AS (
    SELECT 
        discovery_year,
        detection_method,
        COUNT(*) AS method_count
    FROM exoplanets
    GROUP BY discovery_year, detection_method
),
RankedTrends AS (
    SELECT 
        discovery_year,
        detection_method,
        method_count, RANK() OVER (PARTITION BY discovery_year ORDER BY method_count DESC) AS method_rank FROM DetectionTrends) SELECT  discovery_year, detection_method, method_count, method_rank FROM RankedTrends WHERE method_rank <= 2 ORDER BY discovery_year DESC, method_rank;"""

query3 = """WITH DetectionTrends AS (
    SELECT 
        discovery_year,
        detection_method,
        COUNT(*) AS method_count
    FROM exoplanets
    GROUP BY discovery_year, detection_method
),
RankedTrends AS (
    SELECT 
        discovery_year,
        detection_method,
        method_count, RANK() OVER (PARTITION BY discovery_year ORDER BY method_count DESC) AS method_rank FROM DetectionTrends) SELECT  discovery_year, detection_method, method_count, method_rank FROM RankedTrends WHERE method_rank <= 3 ORDER BY discovery_year DESC, method_rank;"""

In [30]:
res1=run_query(query1,cursor)
res1

'[{"discovery_year": 1992, "detection_method": "Pulsar Timing", "method_count": 2, "method_rank": 1}, {"discovery_year": 1994, "detection_method": "Pulsar Timing", "method_count": 1, "method_rank": 1}, {"discovery_year": 1995, "detection_method": "Radial Velocity", "method_count": 1, "method_rank": 1}, {"discovery_year": 1996, "detection_method": "Radial Velocity", "method_count": 6, "method_rank": 1}, {"discovery_year": 1997, "detection_method": "Radial Velocity", "method_count": 1, "method_rank": 1}, {"discovery_year": 1998, "detection_method": "Radial Velocity", "method_count": 6, "method_rank": 1}, {"discovery_year": 1999, "detection_method": "Radial Velocity", "method_count": 13, "method_rank": 1}, {"discovery_year": 2000, "detection_method": "Radial Velocity", "method_count": 16, "method_rank": 1}, {"discovery_year": 2001, "detection_method": "Radial Velocity", "method_count": 12, "method_rank": 1}, {"discovery_year": 2002, "detection_method": "Radial Velocity", "method_count": 2

In [32]:
res2=run_query(query2, cursor)
res2

'[{"discovery_year": 2023, "detection_method": "Transit", "method_count": 6, "method_rank": 1}, {"discovery_year": 2023, "detection_method": "Radial Velocity", "method_count": 3, "method_rank": 2}, {"discovery_year": 2022, "detection_method": "Transit", "method_count": 188, "method_rank": 1}, {"discovery_year": 2022, "detection_method": "Radial Velocity", "method_count": 116, "method_rank": 2}, {"discovery_year": 2021, "detection_method": "Transit", "method_count": 415, "method_rank": 1}, {"discovery_year": 2021, "detection_method": "Radial Velocity", "method_count": 78, "method_rank": 2}, {"discovery_year": 2020, "detection_method": "Transit", "method_count": 166, "method_rank": 1}, {"discovery_year": 2020, "detection_method": "Radial Velocity", "method_count": 45, "method_rank": 2}, {"discovery_year": 2019, "detection_method": "Transit", "method_count": 113, "method_rank": 1}, {"discovery_year": 2019, "detection_method": "Radial Velocity", "method_count": 66, "method_rank": 2}, {"dis

In [53]:
res3=run_query(query3, cursor)
res3

'[{"discovery_year": 2023, "detection_method": "Transit", "method_count": 6, "method_rank": 1}, {"discovery_year": 2023, "detection_method": "Radial Velocity", "method_count": 3, "method_rank": 2}, {"discovery_year": 2022, "detection_method": "Transit", "method_count": 188, "method_rank": 1}, {"discovery_year": 2022, "detection_method": "Radial Velocity", "method_count": 116, "method_rank": 2}, {"discovery_year": 2022, "detection_method": "Gravitational Microlensing", "method_count": 25, "method_rank": 3}, {"discovery_year": 2021, "detection_method": "Transit", "method_count": 415, "method_rank": 1}, {"discovery_year": 2021, "detection_method": "Radial Velocity", "method_count": 78, "method_rank": 2}, {"discovery_year": 2021, "detection_method": "Gravitational Microlensing", "method_count": 20, "method_rank": 3}, {"discovery_year": 2020, "detection_method": "Transit", "method_count": 166, "method_rank": 1}, {"discovery_year": 2020, "detection_method": "Radial Velocity", "method_count":

In [26]:
type(res1)

str

In [45]:
# TODO: Check query equivalence
def parse_results(results_json):
    """Parse the JSON results from the 'results' column."""
    return json.loads(results_json)

def compare_results(query_results, expected_results):
    """Compare the query results with the expected results."""
    # Sort the results for comparison
    query_results_sorted = sorted(query_results, key=lambda x: json.dumps(x, sort_keys=True))
    expected_results_sorted = sorted(expected_results, key=lambda x: json.dumps(x, sort_keys=True))
    return query_results_sorted == expected_results_sorted

def check_query_equivalence(query, expected_results_json, cursor):
    """Check if the query results are equivalent to the expected results."""
    query_results = run_query(query, cursor)
    return compare_results(query_results, expected_results_json)

In [46]:
compare_results(res1,res2)

True

In [47]:
compare_results(res1,res3)

False

In [48]:
check_query_equivalence(query1,res1,cursor)

True

In [49]:
check_query_equivalence(query1,res3,cursor)

False

- Use OperationalError to mark queries that didn't execute; really work on having output only return SQL or parsing out the SQL. 

In [ ]:
import torch; torch.cuda.is_available()

In [6]:

# from premsql.executors.from_sqlite import SQLiteExecutor
# from premsql.datasets import Text2SQLDataset
# from premsql.tuner.peft import Text2SQLPeftTuner
# from premsql.datasets.error_dataset import ErrorDatasetGenerator

## Generate from a ready model

In [55]:
context = """CREATE TABLE exoplanets (
    name TEXT,
    distance REAL,
    stellar_magnitude REAL,
    planet_type TEXT,
    discovery_year INTEGER,
    mass_multiplier REAL,
    mass_wrt TEXT,
    radius_multiplier REAL,
    radius_wrt TEXT,
    orbital_radius REAL,
    orbital_period REAL,
    eccentricity REAL,
    detection_method TEXT
); CREATE TABLE reference_planets (name TEXT, mass REAL);  'mass_wrt' in exoplanets table has a one-to-one match to 'name' in reference_planets table, and  'name' refers to either Earth or Jupyter, with Jupyter having 317.8 the mass of Earth."""

question = """Rank exoplanets based on their mass within each discovery year and return the top 3 planets with the highest mass."""
answer = """WITH RankedExoplanets AS (
    SELECT 
        e.name,
        e.mass_multiplier * r.mass AS mass,
        e.discovery_year,
        RANK() OVER (PARTITION BY e.discovery_year ORDER BY e.mass_multiplier * r.mass DESC) AS mass_rank
    FROM exoplanets e
    JOIN reference_planets r ON e.mass_wrt = r.name
)

SELECT * 
FROM RankedExoplanets
WHERE mass_rank <= 3; """
full_question = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful assistant who is a SQL expert. Your task is to write a SQL query based on the given context and question.<|eot_id|><|start_header_id|>user<|end_header_id|>

Context: {context}
Question: {question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>SQL Query:"""

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
MODEL_ID="premai-io/prem-1B-SQL"
model = AutoModelForCausalLM.from_pretrained(MODEL_ID).to("cuda:1")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)


/home/mainuser/anaconda3/envs/sqlft/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Unrecognized keys in `rope_scaling` for 'rope_type'='linear': {'type'}
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 11.24it/s]


In [4]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32256, 2048)
    (layers): ModuleList(
      (0-23): 24 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5504, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5504, bias=False)
          (down_proj): Linear(in_features=5504, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-06)
      )
    )
    (norm)

In [5]:
tokenizer

LlamaTokenizerFast(name_or_path='premai-io/prem-1B-SQL', vocab_size=32000, model_max_length=16384, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<｜begin▁of▁sentence｜>', 'eos_token': '<|EOT|>', 'pad_token': '<｜end▁of▁sentence｜>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	32000: AddedToken("õ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	32001: AddedToken("÷", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	32002: AddedToken("Á", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	32003: AddedToken("ý", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	32004: AddedToken("À", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	32005: AddedToken("ÿ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	32006: AddedToken("ø", rstrip=False, lst

In [10]:
tokenizer

LlamaTokenizerFast(name_or_path='premai-io/prem-1B-SQL', vocab_size=32000, model_max_length=16384, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<｜begin▁of▁sentence｜>', 'eos_token': '<|EOT|>', 'pad_token': '<｜end▁of▁sentence｜>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	32000: AddedToken("õ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	32001: AddedToken("÷", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	32002: AddedToken("Á", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	32003: AddedToken("ý", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	32004: AddedToken("À", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	32005: AddedToken("ÿ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	32006: AddedToken("ø", rstrip=False, lst

In [17]:
import torch

def generate_sql(context, question, max_new_tokens = 256, debug=False):
    full_question = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful assistant who is a SQL expert. Your task is to write a SQL query based on the given context and question.<|eot_id|><|start_header_id|>user<|end_header_id|>

Context: {context}
Question: {question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>SQL Query:"""
    model_input = tokenizer(full_question, return_tensors="pt").to("cuda:1")
    model.eval()
    with torch.no_grad():
        out = tokenizer.decode(model.generate(**model_input,max_new_tokens=max_new_tokens)[0],skip_special_tokens=True)
    res = out[out.find('SQL Query:')+11:]
    if debug: print(res)
    run_query(res, cursor)
    return res



In [26]:
import torch
# BAD, BAD, BAD
def generate_sql(context, question, max_new_tokens = 256, debug=False):
    full_question = f"""<|begin_of_sentence|>

System: You are a helpful assistant who is a SQL expert. Your task is to write a SQL query based on the given context and question.
<|User|>
Context: {context}
Question: {question}<|EOT|>
<|Assistant|>
SQL Query:<｜fim▁begin｜>"""
    model_input = tokenizer(full_question, return_tensors="pt").to("cuda:1")
    model.eval()
    with torch.no_grad():
        out = tokenizer.decode(model.generate(**model_input,max_new_tokens=max_new_tokens)[0],skip_special_tokens=True)
    res = out[out.find('SQL Query:')+11:]
    if debug: print(res)
    run_query(res, cursor)
    return res



In [12]:
res = out[out.find('SQL Query:')+11:]
print(res)
run_query(res, cursor)

NameError: name 'out' is not defined

- Now generalize it to run on every query in valid set and get the results, counting OperationalError (query valid against db?), and perhaps starting to count execution accuracy (query accurate?)

In [14]:
with open('./databases/valid.json', 'r') as valid_file:
    valid_loaded = json.load(valid_file)
    print("Valid Data Loaded:")
    print(valid_loaded[:2])  # Print the first 2 records for verification

Valid Data Loaded:
[{'context': "CREATE TABLE exoplanets (\n    name TEXT,\n    distance REAL,\n    stellar_magnitude REAL,\n    planet_type TEXT,\n    discovery_year INTEGER,\n    mass_multiplier REAL,\n    mass_wrt TEXT,\n    radius_multiplier REAL,\n    radius_wrt TEXT,\n    orbital_radius REAL,\n    orbital_period REAL,\n    eccentricity REAL,\n    detection_method TEXT\n); CREATE TABLE reference_planets (name TEXT, mass REAL);  'mass_wrt' in exoplanets table has a one-to-one match to 'name' in reference_planets table, and  'name' refers to either Earth or Jupyter, with Jupyter having 317.8 the mass of Earth.", 'question': 'Determine the number of exoplanets discovered each year, and show the year with the highest number of discoveries.', 'answer': 'SELECT discovery_year, COUNT(*) AS count\nFROM exoplanets\nGROUP BY discovery_year\nORDER BY count DESC\nLIMIT 1;', 'results': '[{"discovery_year": 2016, "count": 1517}]'}, {'context': "CREATE TABLE exoplanets (\n    name TEXT,\n    dis

In [15]:
len(valid_loaded)

10

In [27]:
num_opp_errors = 0
num_other_exceptions = 0
for example in valid_loaded:
    try:
        generate_sql(context=example['context'], question=example['question'],debug=True)
    except sqlite3.OperationalError:
        num_opp_errors += 1
    except Exception as e:
        num_other_exceptions +=1
        print(e)
print(f"num_opp_errors: {num_opp_errors}, num_other_exceptions:{num_other_exceptions},\nError rate: {(num_other_exceptions+num_opp_errors)*100/len(valid_loaded)}")

Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


｜fim▁begin｜>SELECT discovery_year, COUNT(*) as num_exoplanets FROM exoplanets GROUP BY discovery_year ORDER BY num_exoplanets DESC LIMIT 1;


Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


｜fim▁begin｜>SELECT detection_method FROM exoplanets WHERE mass_wrt = (SELECT name FROM reference_planets WHERE mass = 317.8) ORDER BY mass DESC LIMIT 1;


Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


...

｜fim▁begin｜>SELECT name, mass FROM (SELECT name, mass, discovery_year, MIN(mass) OVER (PARTITION BY discovery_year) as min_mass FROM exoplanets) WHERE mass = min_mass;


Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


｜fim▁begin｜>SELECT name, discovery_year, detection_method FROM exoplanets WHERE discovery_year BETWEEN (SELECT MAX(discovery_year) - 2 FROM exoplanets) AND MAX(discovery_year);


Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


｜fim▁begin｜>SELECT planet_type, AVG(orbital_radius) AS avg_orbital_radius FROM exoplanets GROUP BY planet_type ORDER BY avg_orbital_radius DESC;
｜fim▁begin｜>SELECT name, (mass_wrt::REAL / radius_wrt::REAL) AS ratio FROM exoplanets ORDER BY ratio DESC LIMIT 5;
num_opp_errors: 10, num_other_exceptions:0,
Error rate: 100.0


### Now see if can convert Gretl data to sqlite format

In [ ]:
#!pip3 install "sqlglot[rs]"

In [ ]:
import sqlglot
sqlglot.transpile("SELECT EPOCH_MS(1618088028295)", read="mysql", write="sqlite")[0]

In [ ]:
help(sqlglot.transpile)

In [ ]:
import sqlglot
sqlglot.transpile("SELECT AVG(word_count) OVER (PARTITION BY EXTRACT(YEAR_QUARTER FROM publish_date)) AS avg_word_count FROM articles WHERE category = 'social_justice' AND location = 'USA' AND YEAR(publish_date) BETWEEN 2021 AND 2022;", read="mysql", write="sqlite")[0]

In [111]:
#!pip install premsql

In [100]:
from premsql.pipelines.simple import SimpleText2SQLAgent
from premsql.generators.huggingface import Text2SQLGeneratorHF
from langchain_community.utilities.sql_database import SQLDatabase
from premsql.utils import convert_sqlite_path_to_dsn

dsn_or_db_path = convert_sqlite_path_to_dsn(
  "exoplanets_db.db"   
)
db = SQLDatabase.from_uri(dsn_or_db_path)

In [101]:
#!pip install tabulate
#!pip install openai

## See if can use llama3b (currently no) and prem-1B-SQL to generate answers by connecting to db

In [ ]:
agent = SimpleText2SQLAgent(
    dsn_or_db_path=db,
    generator=Text2SQLGeneratorHF(
        model_or_name_or_path="premai-io/prem-1B-SQL",
        #model_or_name_or_path="llama3-3b",
        experiment_name="test_nli",
        device="cuda:0",
        type="test"
    ),
)

In [ ]:
response = agent.query(
    question="please list the most recently discovered exoplanet",
    do_sample = True
)

response["table"]

In [ ]:
response

In [ ]:
response = agent.query(
    question="please list the most recently discovered exoplanet",
    do_sample = True
)

response["table"]

In [ ]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Tables in the database:")
for table in tables:
    print(table[0])

## Consider generating more questions from seed of 20 programmatically: TODO

In [ ]:
import random

# Sample dataset of questions and queries
questions = [
    {
        "question": "What is the average mass of exoplanets discovered each year?",
        "query": "SELECT discovery_year, AVG(mass) FROM exoplanets GROUP BY discovery_year;"
    },
    {
        "question": "How many exoplanets were discovered using each method?",
        "query": "SELECT detection_method, COUNT(*) FROM exoplanets GROUP BY detection_method;"
    },
    # Add more questions and queries...
]

# Example of variable options to permute
variables = {
    "threshold": [5, 10, 15],
    "column": ["stellar_magnitude", "distance", "orbital_period"],
    "aggregate": ["AVG", "COUNT", "SUM"],
}

# Function to create permutations
def generate_permutations(questions, variables):
    new_questions = []
    for item in questions:
        for threshold in variables["threshold"]:
            for column in variables["column"]:
                for agg in variables["aggregate"]:
                    # Create a new question and query
                    new_question = item["question"].replace("average mass", f"{agg.lower()} {column}") + f" (threshold: {threshold})"
                    new_query = item["query"].replace("AVG(mass)", f"{agg}({column})").replace("WHERE ...", f"WHERE {column} > {threshold}")
                    
                    new_questions.append({
                        "question": new_question,
                        "query": new_query
                    })
    return new_questions

# Generate new questions and queries
new_dataset = generate_permutations(questions, variables)

# Display new questions and queries
for item in new_dataset:
    print(f"Question: {item['question']}")
    print(f"SQL Query: {item['query']}")
    print()


## TODOs:

- expand data generation from seed (50-100 examples), all tested
- sqglot exector or premai executor (experiment)-> Unit tests
- other ways to write unit tests?  see Hammel
- decide on finetuning approach; with whatever approach must be able to test against db, so see if can test llama3-3b against db or finetune prem in full precision perhaps (just on my data)
- sqlite android/ios integration and emulator testing